In [1]:
#import nesscary libraries
from sklearn.preprocessing import LabelEncoder
import regex as re
import numpy as np 
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

In [2]:
#import data
file = 'D:\datamining\spamnotspam\spam.csv'

In [3]:
import chardet
with open(file, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'Windows-1252', 'confidence': 0.7270322499829184, 'language': ''}

# data exploration

In [4]:
df = pd.read_csv(file,encoding='Windows-1252')

In [5]:
#rename columns
df = df[['v1','v2']]
df.rename(columns= {'v1':"class", 'v2':"Emails"}, inplace = True)

In [6]:
df.columns.name='index'

In [7]:
df.shape

(5572, 2)

In [8]:
df

index,class,Emails
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


# data pre-processing 

In [9]:
def func1(line):
    line = line.lower()
    return line
def func2(line):
    
    line=line.strip()
    line = line.split()
    return line   
def func4(last):
    lsd=[]
    for i in range(len(last)):
        x=re.sub('[^a-z]+','',last[i])
        if x.strip():
            lsd.append(x)
    return lsd
def func5(last):
    sw = ['is', 'are', 'the','i','a','we','am','its','it']
    lsd = filter(lambda x: x not in sw,last)
    return list(lsd)

In [10]:
df['Emails']=df['Emails'].apply(func1)

In [11]:
df['Emails']=df['Emails'].apply(func2)

In [12]:
df['Emails']=df['Emails'].apply(func4)

In [13]:
df['Emails']=df['Emails'].apply(func5)

In [14]:
df['Emails']

0       [go, until, jurong, point, crazy, available, o...
1                          [ok, lar, joking, wif, u, oni]
2       [free, entry, in, wkly, comp, to, win, fa, cup...
3       [u, dun, say, so, early, hor, u, c, already, t...
4       [nah, dont, think, he, goes, to, usf, he, live...
                              ...                        
5567    [this, nd, time, have, tried, contact, u, u, h...
5568            [will, b, going, to, esplanade, fr, home]
5569    [pity, was, in, mood, for, that, soany, other,...
5570    [guy, did, some, bitching, but, acted, like, i...
5571                               [rofl, true, to, name]
Name: Emails, Length: 5572, dtype: object

In [15]:
lec = LabelEncoder()
class_ = ['ham', 'spam']
lec.fit(class_)
df['class']=lec.transform(df['class'])

In [16]:
df

index,class,Emails
0,0,"[go, until, jurong, point, crazy, available, o..."
1,0,"[ok, lar, joking, wif, u, oni]"
2,1,"[free, entry, in, wkly, comp, to, win, fa, cup..."
3,0,"[u, dun, say, so, early, hor, u, c, already, t..."
4,0,"[nah, dont, think, he, goes, to, usf, he, live..."
...,...,...
5567,1,"[this, nd, time, have, tried, contact, u, u, h..."
5568,0,"[will, b, going, to, esplanade, fr, home]"
5569,0,"[pity, was, in, mood, for, that, soany, other,..."
5570,0,"[guy, did, some, bitching, but, acted, like, i..."


# Now to create our vocabulary

In [17]:
vocab = []
for emails in df['Emails']:
    for words in emails:
        vocab.append(words)
vocab = list(set(vocab))

In [18]:
len(vocab)

8529

In [19]:
wpe = {unq : [0] * len(df['Emails']) for unq in vocab}

# This code puts values to the coresponding word in Emails

In [20]:
for index, eml in enumerate(df['Emails']):
   for word in eml:
      wpe[word][index] += 1

# Now creating the data frame 

In [21]:
wpe = pd.DataFrame(wpe)

# Combining the two DFs

In [22]:
df=pd.concat([df,wpe],axis = 1)

In [23]:
df

,class,Emails,forward,gimmi,arng,multis,whens,texting,updatenow,dooms,...,embarassed,internal,nuerologist,drizzling,offering,mobcudb,wikipediacom,ipad,bucks,dehydrated
0,0,"[go, until, jurong, point, crazy, available, o...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,"[ok, lar, joking, wif, u, oni]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,"[free, entry, in, wkly, comp, to, win, fa, cup...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,"[u, dun, say, so, early, hor, u, c, already, t...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,"[nah, dont, think, he, goes, to, usf, he, live...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,1,"[this, nd, time, have, tried, contact, u, u, h...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5568,0,"[will, b, going, to, esplanade, fr, home]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5569,0,"[pity, was, in, mood, for, that, soany, other,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5570,0,"[guy, did, some, bitching, but, acted, like, i...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Train and test 

In [24]:
x=df.iloc[:,2:]
y=df.iloc[:,:1] 

In [25]:
x

,forward,gimmi,arng,multis,whens,texting,updatenow,dooms,sport,snuggles,...,embarassed,internal,nuerologist,drizzling,offering,mobcudb,wikipediacom,ipad,bucks,dehydrated
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5568,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
y.shape

(5572, 1)

In [27]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.3,train_size=0.7,random_state=101)

In [28]:
bnb = BernoulliNB()
ytrain.shape

(3900, 1)

In [29]:
bnb.fit(xtrain,ytrain)

BernoulliNB()

In [30]:
ypdct = bnb.predict(xtest)

In [31]:
acry = round(accuracy_score(ytest, ypdct), 2)

In [32]:
acry

0.97

In [33]:
test = pd.concat([ytest,xtest],axis = 1)

In [34]:
itv = lec.inverse_transform(ypdct)
itv = lec.inverse_transform(ytest)
print(test.assign(predicted_gender = itv))

      class  forward  gimmi  arng  multis  whens  texting  updatenow  dooms  \
2373      1        0      0     0       0      0        0          0      0   
1942      0        0      0     0       0      0        0          0      0   
54        1        0      0     0       0      0        0          0      0   
1900      0        0      0     0       0      0        0          0      0   
1564      0        0      0     0       0      0        0          0      0   
...     ...      ...    ...   ...     ...    ...      ...        ...    ...   
790       0        0      0     0       0      0        0          0      0   
3982      0        0      0     0       0      0        0          0      0   
823       1        0      0     0       0      0        0          0      0   
1911      0        0      0     0       0      0        0          0      0   
1237      0        0      0     0       0      0        0          0      0   

      sport  ...  internal  nuerologist  drizzling 

# Our algorithm is 97% accurate thus can be deployed